In [1]:
# We need Nifty Data for 5 min chart
# Get first 15 min High and low and identify breakout
# Trail Stoploss
# Exit Trade at SL and TSL or EOD

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import datetime
import yfinance as yf
import math
import matplotlib.pyplot as plt


In [2]:
def backTestBreakOut(df, ticker, investment=20000):
    close = df['Close']
    high = df['High']
    low = df['Low']
    open = df['Open']
    date = df.index

    startTime = datetime.time(9, 15)
    
    exitTime = datetime.time(14, 45)
    exitTime


    # initial condition
    in_position = False
    equity = investment

    commission = 10
    share = 0
    tradeTakenForDate = []
    entry = []
    exit = []
    tradeCount=0
    buyTime = ''
    sellTime = ''
    buyPrice = 0
    sellPrice = 0
    trailSL=0
    longTarget=0
    shortTarget=0
    
    buyOrSell=0 # 1 is buy Sell is 0
    #print(f'Testing :  {ticker}, Equity={equity}')
    for i in range(0, len(df)):
        #Check if a trade is already taken for this day

        #intialise High and Low at start of day
        if(date[i].time()==startTime):
            #print(f'StartTime : {startTime}, Date : {date[i]}, in_position : {in_position}')
            startHigh = high[i]
            startLow = low[i]
            longTarget=startHigh+(startHigh-startLow)
            shortTarget=startLow-(startHigh-startLow)

        #now buy of there is High breakout
        elif (not in_position )and close[i]>startHigh and (date[i].date() not in tradeTakenForDate):
            share = math.floor(equity / close[i] )
            equity -= share * close[i]
            buyDate = date[i]
            in_position = True
            buyOrSell=1
            tradeCount += 1
            trailSL=startLow
            tradeTakenForDate.append(date[i].date())
            entry.append((startHigh, buyDate,buyOrSell))
            #print(f'StartHigh : {startHigh}, StartLow : {startLow}, shortTarget : {shortTarget}')
            #print(f'Buy initiated {share} shares at {startHigh} on {date[i]}, longTarget = {longTarget}, shortTarget= {shortTarget}')

        #sell if there is low breakout  
        elif (not in_position )and close[i]<startLow and (date[i].date() not in tradeTakenForDate):
            share = math.floor(equity / close[i] )
            equity += share * close[i]
            sellDate = date[i]
            in_position = True
            buyOrSell=0
            tradeCount += 1
            trailSL=startHigh
            tradeTakenForDate.append(date[i].date())
            exit.append((startLow, sellDate, buyOrSell))
            #print(f'Sell initiated {share} shares at {startLow} on {date[i]} longTarget = {longTarget}, shortTarget= {shortTarget}')
  
        
        #Close positions before EOD
        elif(in_position and date[i].time()==exitTime): 
            #print(f'ExitTime : {exitTime}, Date : {date[i]}')
            if(buyOrSell==1):
                equity += share * close[i] - commission
                sellDate = date[i]
                in_position = False
                exit.append((close[i], sellDate,buyOrSell, equity))
                #print(f'Close buy EOD at {round(close[i],2)} on {date[i]}')
            else:
                equity -= share * close[i] - commission
                buyDate = date[i]
                in_position = False
                entry.append((close[i], buyDate, buyOrSell, equity))
                #print(f'Close Sell EOD at {round(close[i],2)} on {date[i]}')
        
        #Close position if Close hit Target or Stop Loss(no trailing for now)
        elif in_position:
            #print(f'Postion: {buyOrSell} Date : {date[i]}, longTarget : {longTarget}, shortTarget : {shortTarget}, TrailSL : {trailSL}' )
            if buyOrSell==1 :
                if(close[i]>longTarget):
                    equity += share * close[i] - commission
                    sellDate = date[i]
                    in_position = False
                    exit.append((longTarget, sellDate, buyOrSell, equity))
                    #print(f'Close Buy at {longTarget} on {date[i]}')
                    
                elif (close[i]<startLow):
                    equity += share * close[i] - commission
                    sellDate = date[i]
                    in_position = False
                    exit.append((startLow, sellDate, buyOrSell, equity))
                    #print(f'Close Buy at {longTarget} on {date[i]}')
                
            elif buyOrSell==0:
                if(close[i]<shortTarget):
                    equity -= share * close[i] - commission
                    buyDate = date[i]
                    in_position = False
                    entry.append((shortTarget, buyDate, buyOrSell, equity))
                    #print(f'Close Sell at {shortTarget} on {date[i]}')
                elif (close[i]>startHigh):
                    equity -= share * close[i] - commission
                    buyDate = date[i]
                    in_position = False
                    entry.append((startHigh, buyDate, buyOrSell, equity))
                    #print(f'Close Sell at {shortTarget} on {date[i]}')

        
        if in_position:
            if(buyOrSell==1) and close[i]>trailSL:
                trailSL = low[i]               
            elif(buyOrSell==0) and close[i]<trailSL:
                trailSL = high[i]               
        

    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from {ticker} investing {investment} is ${round(earning,2)} \t (ROI = {roi}%) \t Trades={tradeCount}')
    return entry, exit, earning, roi, tradeCount

In [3]:
def backTestBreakOutWithTrailSL(df, ticker, investment=20000):
    close = df['Close']
    high = df['High']
    low = df['Low']
    open = df['Open']
    date = df.index

    startTime = datetime.time(9, 15)
    
    exitTime = datetime.time(14, 45)
    exitTime


    # initial condition
    in_position = False
    equity = investment

    commission = 10
    share = 0
    tradeTakenForDate = []
    entry = []
    exit = []
    tradeCount=0
    buyTime = ''
    sellTime = ''
    buyPrice = 0
    sellPrice = 0
    trailSL=0
    longTarget=0
    shortTarget=0
    
    buyOrSell=0 # 1 is buy Sell is 0
    #print(f'Testing :  {ticker}, Equity={equity}')
    for i in range(0, len(df)):
        #Check if a trade is already taken for this day

        #intialise High and Low at start of day
        if(date[i].time()==startTime):
            #print(f'StartTime : {startTime}, Date : {date[i]}, in_position : {in_position}')
            startHigh = high[i]
            startLow = low[i]
            longTarget=startHigh+(startHigh-startLow)
            shortTarget=startLow-(startHigh-startLow)

        #now buy of there is High breakout
        elif (not in_position )and close[i]>startHigh and (date[i].date() not in tradeTakenForDate):
            share = math.floor(equity / close[i] )
            equity -= share * close[i]
            buyDate = date[i]
            in_position = True
            buyOrSell=1
            tradeCount += 1
            trailSL=longTarget
            tradeTakenForDate.append(date[i].date())
            entry.append((startHigh, buyDate,buyOrSell))
            #print(f'StartHigh : {startHigh}, StartLow : {startLow}, shortTarget : {shortTarget}')
            #print(f'Buy initiated {share} shares at {startHigh} on {date[i]}, longTarget = {longTarget}, shortTarget= {shortTarget}')

        #sell if there is low breakout  
        elif (not in_position )and close[i]<startLow and (date[i].date() not in tradeTakenForDate):
            share = math.floor(equity / close[i] )
            equity += share * close[i]
            sellDate = date[i]
            in_position = True
            buyOrSell=0
            tradeCount += 1
            trailSL=shortTarget
            tradeTakenForDate.append(date[i].date())
            exit.append((startLow, sellDate, buyOrSell))
            #print(f'Sell initiated {share} shares at {startLow} on {date[i]} longTarget = {longTarget}, shortTarget= {shortTarget}')
  
        
        #Close positions before EOD
        elif(in_position and date[i].time()==exitTime): 
            #print(f'ExitTime : {exitTime}, Date : {date[i]}')
            if(buyOrSell==1):
                equity += share * close[i] - commission
                sellDate = date[i]
                in_position = False
                exit.append((close[i], sellDate,buyOrSell, equity))
                #print(f'Close buy EOD at {round(close[i],2)} on {date[i]}')
            else:
                equity -= share * close[i] - commission
                buyDate = date[i]
                in_position = False
                entry.append((close[i], buyDate, buyOrSell, equity))
                #print(f'Close Sell EOD at {round(close[i],2)} on {date[i]}')
        
        #Close position if Close hit Target or Stop Loss(no trailing for now)
        elif in_position:
            #print(f'Postion: {buyOrSell} Date : {date[i]}, longTarget : {longTarget}, shortTarget : {shortTarget}, TrailSL : {trailSL}' )
            if buyOrSell==1 :
                if(close[i]<trailSL and trailSL!=longTarget):
                    equity += share * close[i] - commission
                    sellDate = date[i]
                    in_position = False
                    exit.append((trailSL, sellDate, buyOrSell, equity))
                    #print(f'Close Buy at {longTarget} on {date[i]}')
                    
                elif (close[i]<startLow):
                    equity += share * close[i] - commission
                    sellDate = date[i]
                    in_position = False
                    exit.append((startLow, sellDate, buyOrSell, equity))
                    #print(f'Close Buy at {longTarget} on {date[i]}')
                
            elif buyOrSell==0:
                if(close[i]>trailSL and trailSL!=shortTarget ):
                    equity -= share * close[i] - commission
                    buyDate = date[i]
                    in_position = False
                    entry.append((shortTarget, buyDate, buyOrSell, equity))
                    #print(f'Close Sell at {shortTarget} on {date[i]}')
                elif (close[i]>startHigh):
                    equity -= share * close[i] - commission
                    buyDate = date[i]
                    in_position = False
                    entry.append((startHigh, buyDate, buyOrSell, equity))
                    #print(f'Close Sell at {shortTarget} on {date[i]}')

        
        if in_position:
            if(buyOrSell==1) and close[i]>longTarget:
                trailSL = open[i]               
            elif(buyOrSell==0) and close[i]<shortTarget:
                trailSL = open[i]               
        

    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from {ticker} investing {investment} is ${round(earning,2)} \t (ROI = {roi}%) \t Trades={tradeCount}')
    return entry, exit, earning, roi, tradeCount

In [4]:
def breakoutCheck(df, ticker, investment=20000):
    entry, exit, earning, roi, tradeCount = backTestBreakOut(df, ticker, investment)
    entryDF = pd.DataFrame(entry,columns=['BuyPrice','BuyDate','BuySell','Equity'])
    exitDF = pd.DataFrame(exit,columns=['SellPrice','SellDate','BuySell2','Equity'])
    result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
    result['PnL_Points'] = (result['SellPrice'] - result['BuyPrice'])
    PnLSum=sum(result['PnL_Points'])
    AvgProfit=result['PnL_Points'].mean()
    #print(f'TradeCount : {tradeCount}, Sum = {PnLSum}, AvgProfit = {AvgProfit}')
    result['FinalReturn']= np.where(result.BuySell==1,result.Equity_y,result.Equity_x)
    result['Symbol'] = ticker
    result.drop(['BuySell2','Equity_x','Equity_y'], axis=1, inplace=True)
    return result

In [5]:
def NSEData(tickerList):
    

    #Fetch NSE Ticker Data
    result=[] 
    for ticker in tickerList: 
        data = []
        symbol = ticker + ".NS" 
        
        d = dt.today() - datetime.timedelta(days=50)
        startDate = d.date().strftime("%Y-%m-%d")
        df = yf.download(symbol, start=startDate,interval='15m') 

        
        data.append(symbol)
        data.append(df)
        result.append(data)

    return result    

In [6]:
ticker = 'AARTIIND.BO'
d = dt.today() - datetime.timedelta(days=50)
startDate = d.date().strftime("%Y-%m-%d")
df = yf.download(ticker, start=startDate,interval='15m') 


[*********************100%***********************]  1 of 1 completed


In [7]:

entry, exit,earning, roi, tradeCount = backTestBreakOut(df, ticker, 20000)
entryDF = pd.DataFrame(entry,columns=['BuyPrice','BuyDate','BuySell','Equity'])
exitDF = pd.DataFrame(exit,columns=['SellPrice','SellDate','BuySell2','Equity'])
result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
result['PnL_Points'] = (result['SellPrice'] - result['BuyPrice'])
PnLSum=sum(result['PnL_Points'])
AvgProfit=result['PnL_Points'].mean()
#print(f'TradeCount : {tradeCount}, Sum = {PnLSum}, AvgProfit = {AvgProfit}')
result['FinalReturn']= np.where(result.BuySell==1,result.Equity_y,result.Equity_x)
result['Symbol'] = ticker
result.drop(['BuySell2','Equity_x','Equity_y'], axis=1, inplace=True)

result['PnL_Percent'] = (result['SellPrice'] - result['BuyPrice'])/result['BuyPrice']*100
result['Symbol'] = ticker

result = result[['Symbol','BuyDate', 'BuyPrice', 'SellDate', 'SellPrice','PnL_Percent','FinalReturn','PnL_Points']]
result.to_csv('result/IntradayBreakOut_NSE.csv')


Earning from AARTIIND.BO investing 20000 is $1302.36 	 (ROI = 6.51%) 	 Trades=34


In [9]:
#Get all Trades for given input
# 1. Download Data


#BlueChip Trend from TV is giving more returns
tickerList=['IOB', 'NHPC', 'UNIONBANK', 'IDBI', 'PNB', 'IDFCFIRSTB', 'BHEL', 'BANKINDIA', 'BANKBARODA', 'L&TFH', 'ADANIPOWER', 'TATAPOWER', 'NTPC', 'IOC', 'ONGC', 'SAIL', 'ABCAPITAL', 'ASHOKLEY', 'COALINDIA', 'CANBK', 'TATAMTRDVR', 'GAIL', 'NMDC', 'MOTHERSUMI', 'VEDL', 'HINDPETRO', 'TATAMOTORS', 'AMBUJACEM', 'HINDZINC', 'HINDALCO', 'JINDALSTEL', 'CROMPTON', 'SBIN', 'TORNTPOWER', 'LICHSGFIN', 'MARICO', 'BPCL', 'GUJGASLTD', 'IGL', 'LAURUSLABS', 'BHARTIARTL', 'DABUR', 'ICICIPRULI', 'WIPRO', 'CHOLAFIN', 'SUNTV', 'MCDOWELL-N', 'CADILAHC', 'TATACONSUM', 'CONCOR', 'ICICIBANK', 'TVSMOTOR', 'HDFCLIFE', 'BHARATFORG', 'JSWSTEEL', 'SUNPHARMA', 'AXISBANK', 'ADANIPORTS', 'CUMMINSIND', 'BERGEPAINT', 'UPL', 'M&M', 'TRENT', 'GODREJCP', 'CIPLA', 'RAMCOCEM', 'HCLTECH', 'SBILIFE', 'AUBANK', 'INDUSINDBK', 'VOLTAS', 'HAVELLS', 'AUROPHARMA', 'SBICARD', 'TECHM', 'TATASTEEL', 'LUPIN', 'MUTHOOTFIN', 'ADANIGREEN', 'ADANIENT', 'GODREJPROP', 'ATGL', 'INFY', 'GRASIM', 'ADANITRANS', 'SRTRANSFIN', 'HDFCBANK', 'LT', 'TITAN', 'AARTIIND', 'PEL', 'KOTAKBANK', 'DEEPAKNTR', 'IRCTC', 'ACC', 'PIDILITIND', 'HINDUNILVR', 'MINDTREE', 'HDFC', 'EICHERMOT', 'ASIANPAINT']

nseDF = NSEData(tickerList)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [60]:
nseDF[0]

['IOB.NS',
                                 Open       High        Low      Close  \
 Datetime                                                                
 2021-10-27 09:15:00+05:30  22.350000  22.450001  22.200001  22.250000   
 2021-10-27 09:30:00+05:30  22.299999  22.350000  22.150000  22.350000   
 2021-10-27 09:45:00+05:30  22.299999  22.600000  22.250000  22.450001   
 2021-10-27 10:00:00+05:30  22.500000  22.549999  22.400000  22.450001   
 2021-10-27 10:15:00+05:30  22.400000  22.500000  22.350000  22.350000   
 ...                              ...        ...        ...        ...   
 2021-12-16 14:30:00+05:30  21.049999  21.049999  21.000000  21.000000   
 2021-12-16 14:45:00+05:30  21.049999  21.049999  21.000000  21.049999   
 2021-12-16 15:00:00+05:30  21.049999  21.100000  21.000000  21.100000   
 2021-12-16 15:15:00+05:30  21.100000  21.200001  21.000000  21.150000   
 2021-12-16 15:30:00+05:30  21.100000  21.100000  21.100000  21.100000   
 
                         

In [19]:
# 2. Generate Calls

Earnings = []
ROI =[]
tickerList =[]
investment = []
totalTrades = []
finalResults = []

for data in nseDF:
    df=data[1]
    ticker=data[0]
    df['Date'] = df.index
    
    entry, exit,earning, roi, tradeCount = backTestBreakOut(df, ticker, 20000)
    entryDF = pd.DataFrame(entry,columns=['BuyPrice','BuyDate','BuySell','Equity'])
    exitDF = pd.DataFrame(exit,columns=['SellPrice','SellDate','BuySell2','Equity'])
    result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
    result['PnL_Points'] = (result['SellPrice'] - result['BuyPrice'])
    PnLSum=sum(result['PnL_Points'])
    AvgProfit=result['PnL_Points'].mean()
    #print(f'TradeCount : {tradeCount}, Sum = {PnLSum}, AvgProfit = {AvgProfit}')
    result['FinalReturn']= np.where(result.BuySell==1,result.Equity_y,result.Equity_x)
    result['Symbol'] = ticker
    result.drop(['BuySell2','Equity_x','Equity_y'], axis=1, inplace=True)
    
    result['PnL_Percent'] = (result['SellPrice'] - result['BuyPrice'])/result['BuyPrice']*100
    result['Symbol'] = ticker
    finalResults.append(result)
    
    Earnings.append(earning)
    ROI.append(roi)
    investment.append(20000)
    totalTrades.append(tradeCount)
    tickerList.append(ticker)
    
resultDF = pd.DataFrame(
    {'Ticker': tickerList,
     'Investment': investment,
     'Earnings' : Earnings,
     'FinalReturns': ROI,
     'TotalTrades': totalTrades
    })

finalResults = pd.concat(finalResults)
finalResults
finalResults = finalResults[['Symbol','BuyDate', 'BuyPrice', 'SellDate', 'SellPrice','PnL_Percent','FinalReturn']]
#finalResults.to_excel('SuperTrend_BackTest_Trades.xlsx')
#resultDF.to_excel('SuperTrend_Final_Result.xlsx')


Earning from IOB.NS investing 20000 is $-19995.35 	 (ROI = -99.98%) 	 Trades=32
Earning from NHPC.NS investing 20000 is $18307.35 	 (ROI = 91.54%) 	 Trades=32
Earning from UNIONBANK.NS investing 20000 is $2303.6 	 (ROI = 11.52%) 	 Trades=35
Earning from IDBI.NS investing 20000 is $-19955.1 	 (ROI = -99.78%) 	 Trades=35
Earning from PNB.NS investing 20000 is $25402.3 	 (ROI = 127.01%) 	 Trades=36
Earning from IDFCFIRSTB.NS investing 20000 is $637.6 	 (ROI = 3.19%) 	 Trades=36
Earning from BHEL.NS investing 20000 is $155.65 	 (ROI = 0.78%) 	 Trades=36
Earning from BANKINDIA.NS investing 20000 is $-19982.1 	 (ROI = -99.91%) 	 Trades=32
Earning from BANKBARODA.NS investing 20000 is $-171.65 	 (ROI = -0.86%) 	 Trades=34
Earning from L&TFH.NS investing 20000 is $339.56 	 (ROI = 1.7%) 	 Trades=34
Earning from ADANIPOWER.NS investing 20000 is $1872.15 	 (ROI = 9.36%) 	 Trades=33
Earning from TATAPOWER.NS investing 20000 is $-19903.85 	 (ROI = -99.52%) 	 Trades=33
Earning from NTPC.NS investing

In [70]:
finalResults.to_csv('IntradayBreakout_BackTest_Trades.csv')
resultDF.to_csv('IntradayBreakout_Final_Result.csv')